In [1]:
import nibabel as nib
import numpy as np
from sklearn.metrics import confusion_matrix
import os


def load_nifti(ruta_archivo):
    return nib.load(ruta_archivo).get_fdata() 

datadir_manual = "c:/Users/triple_aaa/Desktop/Tetris/Airways"
datadir_auto = "c:/Users/triple_aaa/Desktop/Tetris/results_w"

path_manual = [os.path.join(datadir_manual, file) for file in os.listdir(datadir_manual)]
path_auto = [os.path.join(datadir_auto, file) for file in os.listdir(datadir_auto)]

In [2]:
gt_data = load_nifti(path_manual[0]).astype(bool)
pred_data = load_nifti(path_auto[0]).astype(bool)
if gt_data.shape == pred_data.shape:
    print(F'LAS DIMENSIONES COINCIDEN')
else:
    print("Verificar dimensiones")

LAS DIMENSIONES COINCIDEN


In [3]:
# Aplanar los arrays para la matriz de confusión
gt_flat = gt_data.flatten()
pred_flat = pred_data.flatten()

# Calcular matriz de confusión
tn, fp, fn, tp = confusion_matrix(gt_flat, pred_flat).ravel()

print(f"Verdaderos negativos (TN): {tn}")
print(f"Falsos positivos (FP): {fp}")
print(f"Falsos negativos (FN): {fn}")
print(f"Verdaderos positivos (TP): {tp}")

Verdaderos negativos (TN): 177834894
Falsos positivos (FP): 8309
Falsos negativos (FN): 35629
Verdaderos positivos (TP): 116944


In [9]:
# Calcular métricas comunes
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)  # También llamado valor predictivo positivo
recall = tp / (tp + fn)     # También llamado sensibilidad
specificity = tn / (tn + fp)
dice_score = 2 * tp / (2 * tp + fp + fn)  # Similar a F1
fpr = fp / (fp + tn)
iou = tp / (tp+fp+fn)

print(f"\nPrecisión: {precision*100:.1f}%")
print(f"Sensibilidad/Recall: {recall*100:.1f}%")
print(f"Especificidad: {specificity*100:.4f}%")
print(f"Dice Score: {dice_score*100:.1f}%")
print(f"Exactitud: {accuracy*100:.1f}%")
print(f"Tasa de Falsos Positivos: {fpr*100:.4f}%")
print(f"Intersección sobre Unión: {iou*100:.1f}%")


Precisión: 93.4%
Sensibilidad/Recall: 76.6%
Especificidad: 99.9953%
Dice Score: 84.2%
Exactitud: 100.0%
Tasa de Falsos Positivos: 0.0047%
Intersección sobre Unión: 72.7%


In [10]:
import pandas as pd
import nibabel as nib
import numpy as np
from sklearn.metrics import confusion_matrix
import os

def load_nifti(ruta_archivo):
    return nib.load(ruta_archivo).get_fdata() 

def calculo_metricas_segmentacion(list_manual, list_auto):

    # Inicializar lista para almacenar resultados
    resultados = []
    metricas_por_caso = []

    for i, (gt_path, pred_path) in enumerate(zip(list_manual, list_auto)):
        try:
            gt_data = load_nifti(gt_path).astype(bool)
            pred_data = load_nifti(pred_path).astype(bool)

            # Verificar dimensiones
            if gt_data.shape != pred_data.shape:
                print(f"Advertencia: Dimensiones no coinciden en par {i+1}")
                print(f"Ground Truth: {gt_data.shape}, Predict: {pred_data.shape} - Saltando...")
                continue
            # Calcular matriz de confusión
            tn, fp, fn, tp = confusion_matrix(
                gt_data.flatten(), 
                pred_data.flatten()
            ).ravel()
            # Calcular métricas
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
            accuracy = (tp + tn) / (tp + tn + fp + fn)
            dice = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0
            fpr = fp / (fp + tn) 
            iou = tp / (tp+fp+fn) 
            
            # Almacenar resultados
            caso = {
                'Caso': os.path.basename(gt_path),
                'TP': tp,
                'FP': fp,
                'FN': fn,
                'TN': tn,
                'Precisión': precision,
                'Recall': recall,
                'Especificidad': specificity,
                'Exactitud': accuracy,
                'Dice': dice,
                'FPR': fpr,
                'IoU': iou
            }
            resultados.append(caso)
            metricas_por_caso.append([precision, recall, specificity, accuracy, dice])
            
        except Exception as e:
            print(f"Error procesando par {i+1}: {str(e)}")
            continue 

    # Crear DataFrame con todos los resultados
    df = pd.DataFrame(resultados)
    
    # Calcular estadísticas globales si hay resultados
    if len(resultados) == 0:
        return {"error": "No se procesaron casos válidos"}, pd.DataFrame()
    
    # Métricas para las cuales calcular estadísticas
    metricas = ['Precisión', 'Recall', 'Especificidad', 'Exactitud','Dice', "FPR", "IoU"]
    
    # Calcular estadísticas
    stats = {}
    for metrica in metricas:
        valores = df[metrica]
        stats[metrica] = {
            'promedio': np.mean(valores),
            'min': np.min(valores),
            'max': np.max(valores),
            'std': np.std(valores),
        }
    
    # Calcular estadísticas para FP, FN
    for metrica in ['TN','FP','FN',"TP"]:
        valores = df[metrica]
        stats[metrica] = {
            'total': np.sum(valores),
            'promedio': np.mean(valores),
            'min': np.min(valores),
            'max': np.max(valores),
            'std': np.std(valores),
        }
    
    return stats, df

In [12]:
def load_nifti(ruta_archivo):
    return nib.load(ruta_archivo).get_fdata() 

datadir_manual = "c:/Users/triple_aaa/Desktop/Tetris/Airways"
datadir_auto = "c:/Users/triple_aaa/Desktop/Tetris/results_w"

path_manual = [os.path.join(datadir_manual, file) for file in os.listdir(datadir_manual)]
path_auto = [os.path.join(datadir_auto, file) for file in os.listdir(datadir_auto)]

stats, df = calculo_metricas_segmentacion(path_manual, path_auto)

In [13]:
df

,Caso,TP,FP,FN,TN,Precisión,Recall,Especificidad,Exactitud,Dice,FPR,IoU
0,ATM_001_0000.nii.gz,116944,8309,35629,177834894,0.933662,0.766479,0.999953,0.999753,0.841851,0.000047,0.726893
1,ATM_002_0000.nii.gz,85576,6758,37648,166331458,0.926809,0.694475,0.999959,0.999733,0.793995,0.000041,0.658368
2,ATM_003_0000.nii.gz,114546,20282,55619,209262609,0.849571,0.673147,0.999903,0.999638,0.751139,0.000097,0.601459
3,ATM_004_0000.nii.gz,152377,22301,72066,177749032,0.872331,0.678912,0.999875,0.999470,0.763563,0.000125,0.617551
4,ATM_005_0000.nii.gz,112028,7092,33998,194095586,0.940463,0.767178,0.999963,0.999788,0.845029,0.000037,0.731645
5,ATM_006_0000.nii.gz,79519,15386,39398,177861473,0.837880,0.668693,0.999914,0.999692,0.743787,0.000086,0.592087
6,ATM_007_0000.nii.gz,197946,14557,51667,209188886,0.931497,0.793012,0.999930,0.999684,0.856694,0.000070,0.749313
7,ATM_008_0000.nii.gz,74390,9289,30823,177881274,0.888992,0.707042,0.999948,0.999775,0.787646,0.000052,0.649683
8,ATM_009_0000.nii.gz,83655,16809,31711,209320881,0.832686,0.725127,0.999920,0.999768,0.775193,0.000080,0.632911
9,ATM_010_0000.nii.gz,130581,29406,47844,209245225,0.816198,0.731854,0.999859,0.999631,0.771728,0.000141,0.628304


In [14]:
# Después de calcular las métricas (al final del ejemplo de uso)
df.to_csv('resultados_metricas.csv', index=False)

In [15]:
df_estadisticas = pd.DataFrame.from_dict(stats, orient='index')
df_estadisticas.to_csv('estadisticas_resumidas.csv')
df_estadisticas

,promedio,min,max,std,total
Precisión,8.670022e-01,7.832033e-01,9.494444e-01,4.931899e-02,NaN
Recall,7.123469e-01,6.060750e-01,8.295953e-01,5.396495e-02,NaN
Especificidad,9.999053e-01,9.998192e-01,9.999697e-01,4.734365e-05,NaN
Exactitud,9.996611e-01,9.994160e-01,9.998880e-01,1.079011e-04,NaN
Dice,7.807038e-01,7.075607e-01,8.801055e-01,4.199392e-02,NaN
FPR,9.472557e-05,3.029099e-05,1.807928e-04,4.734365e-05,NaN
IoU,6.422847e-01,5.474615e-01,7.858826e-01,5.794974e-02,NaN
TN,1.897343e+08,1.611652e+08,2.093209e+08,1.620491e+07,5.692029e+09
FP,1.819047e+04,4.882000e+03,3.782700e+04,9.562762e+03,5.457140e+05
FN,4.639113e+04,1.317600e+04,7.883000e+04,1.473180e+04,1.391734e+06


In [30]:
metrics = df[["Precisión",	"Recall", "Especificidad", "Exactitud", "Dice",	"FPR", "IoU"]]

metrics.mean()*100

Precisión        86.700216
Recall           71.234688
Especificidad    99.990527
Exactitud        99.966111
Dice             78.070376
FPR               0.009473
IoU              64.228470
dtype: float64

In [31]:
metrics.std()*100

Precisión        5.016211
Recall           5.488750
Especificidad    0.004815
Exactitud        0.010975
Dice             4.271182
FPR              0.004815
IoU              5.894040
dtype: float64

In [34]:
metrics.min()*100

Precisión        78.320330
Recall           60.607499
Especificidad    99.981921
Exactitud        99.941598
Dice             70.756075
FPR               0.003029
IoU              54.746151
dtype: float64

In [35]:
metrics.max()*100

Precisión        94.944437
Recall           82.959534
Especificidad    99.996971
Exactitud        99.988799
Dice             88.010555
FPR               0.018079
IoU              78.588259
dtype: float64